<a href="https://colab.research.google.com/github/LearningHarder/deeplearning-course/blob/master/final-pj/SemanticSegmentation/ss_zhangli.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# Install PyTorch
!conda install pytorch=1.10.0 torchvision cudatoolkit=11.1 -c pytorch
# Install MMCV
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10/index.html

/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10/index.html
     |████████████████████████████████| 46.4 MB 9.1 MB/s 
     |████████████████████████████████| 190 kB 4.8 MB/s 


In [ ]:
!rm -rf mmsegmentation
!git clone https://github.com/open-mmlab/mmsegmentation.git 
%cd mmsegmentation
!pip install -e .

Cloning into 'mmsegmentation'...
remote: Enumerating objects: 7242, done.
remote: Counting objects: 100% (247/247), done.
remote: Compressing objects: 100% (198/198), done.
remote: Total 7242 (delta 85), reused 180 (delta 49), pack-reused 6995
Receiving objects: 100% (7242/7242), 13.45 MiB | 33.59 MiB/s, done.
Resolving deltas: 100% (5278/5278), done.
/content/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmsegmentation
     |████████████████████████████████| 577 kB 5.0 MB/s 
  Running setup.py develop for mmsegmentation


In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMSegmentation installation
import mmseg
print(mmseg.__version__)

1.11.0+cu113 True
0.25.0


In [ ]:
# !mkdir checkpoints
# !wget https://download.openmmlab.com/mmsegmentation/v0.5/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth -P checkpoints

--2022-06-03 04:50:49--  https://download.openmmlab.com/mmsegmentation/v0.5/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.28
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196205945 (187M) [application/octet-stream]
Saving to: ‘checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth’

pspnet_r50-d8_512x1 100%[===================>] 187.12M  9.21MB/s    in 21s     

2022-06-03 04:51:11 (8.92 MB/s) - ‘checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth’ saved [196205945/196205945]



In [ ]:
!wget https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r18-d8_512x1024_80k_cityscapes/deeplabv3plus_r18-d8_512x1024_80k_cityscapes_20201226_080942-cff257fe.pth

--2022-06-03 06:32:14--  https://download.openmmlab.com/mmsegmentation/v0.5/deeplabv3plus/deeplabv3plus_r18-d8_512x1024_80k_cityscapes/deeplabv3plus_r18-d8_512x1024_80k_cityscapes_20201226_080942-cff257fe.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.252.96.28
Connecting to download.openmmlab.com (download.openmmlab.com)|47.252.96.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49983942 (48M) [application/octet-stream]
Saving to: ‘deeplabv3plus_r18-d8_512x1024_80k_cityscapes_20201226_080942-cff257fe.pth’

deeplabv3plus_r18-d 100%[===================>]  47.67M  8.86MB/s    in 12s     

2022-06-03 06:32:27 (4.00 MB/s) - ‘deeplabv3plus_r18-d8_512x1024_80k_cityscapes_20201226_080942-cff257fe.pth’ saved [49983942/49983942]



In [ ]:
# build the model from a config file and a checkpoint file
import cv2

from mmseg.apis import inference_segmentor, init_segmentor
from mmseg.core.evaluation import get_palette
# config_file = 'configs/pspnet/pspnet_r50-d8_512x1024_40k_cityscapes.py'
# checkpoint_file = 'checkpoints/pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth'
config_file = 'configs/deeplabv3plus/deeplabv3plus_r18-d8_512x1024_80k_cityscapes.py'
checkpoint_file = 'deeplabv3plus_r18-d8_512x1024_80k_cityscapes_20201226_080942-cff257fe.pth'
output_file = '../result.mp4'
video = "../test.mp4"

model = init_segmentor(config_file, checkpoint_file, device='cuda:0')

# build input video
cap = cv2.VideoCapture(video)
assert (cap.isOpened())
input_height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
input_width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
input_fps = cap.get(cv2.CAP_PROP_FPS)

# init output video
writer = None
output_height = None
output_width = None
if output_file is not None:
  fourcc = cv2.VideoWriter_fourcc('M', 'P', '4', 'V')
  output_fps = input_fps
  output_height = int(
      input_height)
  output_width =  int(
      input_width)
  writer = cv2.VideoWriter(output_file, fourcc, output_fps,
                            (output_width, output_height), True)

# start looping
try:
  while True:
    flag, frame = cap.read()
    if not flag:
        break

    # test a single image
    result = inference_segmentor(model, frame)

    # blend raw image and prediction
    draw_img = model.show_result(
        frame,
        result,
        palette=get_palette('cityscapes'),
        show=False,
        opacity=0.5)

    if writer:
      if draw_img.shape[0] != output_height or draw_img.shape[
              1] != output_width:
        draw_img = cv2.resize(draw_img,(output_width, output_height))
      writer.write(draw_img)
finally:
  if writer:
      writer.release()
  cap.release()

/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '


load checkpoint from local path: deeplabv3plus_r18-d8_512x1024_80k_cityscapes_20201226_080942-cff257fe.pth


/content/mmsegmentation/mmseg/models/segmentors/base.py:289: UserWarning: show==False and out_file is not specified, only result image will be returned
  warnings.warn('show==False and out_file is not specified, only '
